In [1]:
!pip install xgboost
!pip install cartopy shapely pyproj geos

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 8.4 MB/s eta 0:00:00:00:01


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report
from scipy.stats import pearsonr
from xgboost import XGBRegressor

In [3]:
# Mô tả: Định nghĩa các đường dẫn, kích thước bản đồ, và tạo thư mục đầu ra
OUTPUT_PATH = "/kaggle/working/"
BEST_MODEL_PLOTS_PATH = os.path.join(OUTPUT_PATH, "best_model_plots")
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(BEST_MODEL_PLOTS_PATH, exist_ok=True)
CSV_PATH = "/kaggle/input/data-full-features-ai/weather_data_nghean (1).csv"
HEIGHT, WIDTH = 90, 250


In [4]:
# Mô tả: Đọc file CSV, chọn đặc trưng, xử lý NaN, thêm cột hour, và chia dữ liệu
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File {CSV_PATH} không tồn tại!")

print("Đang đọc dữ liệu từ CSV...")
df = pd.read_csv(CSV_PATH)
if df.empty:
    raise ValueError("File CSV rỗng!")

exclude_cols = ['Radar', 'datetime', 'x', 'y']
SELECTED_FEATURES = [col for col in df.columns if col not in exclude_cols]
print("Các cột đặc trưng được chọn từ CSV:", SELECTED_FEATURES)

required_cols = SELECTED_FEATURES + ['Radar', 'datetime', 'x', 'y']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise ValueError(f"Các cột thiếu trong dữ liệu: {missing_cols}")

if df[SELECTED_FEATURES + ['Radar']].isna().any().any():
    print("Cảnh báo: Dữ liệu chứa giá trị NaN. Điền bằng trung bình cho đặc trưng và 0 cho Radar...")
    df[SELECTED_FEATURES] = df[SELECTED_FEATURES].fillna(df[SELECTED_FEATURES].mean())
    df['Radar'] = df['Radar'].fillna(0)

df['datetime'] = pd.to_datetime(df['datetime'])
if 'hour' not in df.columns:
    df['hour'] = df['datetime'].dt.hour
    if 'hour' not in SELECTED_FEATURES:
        SELECTED_FEATURES.append('hour')

X = df[SELECTED_FEATURES]
y = df['Radar']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Đang đọc dữ liệu từ CSV...
Các cột đặc trưng được chọn từ CSV: ['B04B', 'B05B', 'B06B', 'B09B', 'B10B', 'B11B', 'B12B', 'B14B', 'B16B', 'I2B', 'I4B', 'IRB', 'VSB', 'WVB', 'CAPE', 'CIN', 'EWSS', 'IE', 'ISOR', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SLHF', 'SLOR', 'SSHF', 'TCLW', 'TCW', 'TCWV', 'U250', 'U850', 'V250', 'V850']


In [5]:
# Mô tả: Hàm để tính chỉ số đánh giá, vẽ bản đồ lượng mưa, biểu đồ phân tán, và tầm quan trọng đặc trưng
def plot_and_save_results(model, model_name, params, y_train_pred, y_test_pred, X_train, X_test, y_train, y_test, df, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_cc, _ = pearsonr(y_test, y_test_pred)
    
    print(f"\nĐánh giá {model_name} với tham số {params}:")
    print(f"Train RMSE: {train_rmse:.4f}, Train MAE: {train_mae:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}")
    print(f"Test CC: {test_cc:.4f}")
    print(f"R2 Score (Test): {test_r2:.4f}")

    valid_dt = None
    for _ in range(10):
        sample_dt = df['datetime'].sample(1).iloc[0]
        sample_df = df[df['datetime'] == sample_dt].copy()
        if not sample_df.empty:
            valid_dt = sample_dt
            break
    if valid_dt is None:
        print("Cảnh báo: Không tìm thấy thời điểm hợp lệ để vẽ bản đồ lượng mưa")
        return None, None
    
    test_pred_dict = dict(zip(X_test.index, y_test_pred))
    sample_df['prediction'] = sample_df.index.map(test_pred_dict)

    ground_truth_map = np.zeros((HEIGHT, WIDTH))
    prediction_map = np.zeros((HEIGHT, WIDTH))
    for _, row in sample_df.iterrows():
        row_idx = int((df['y'].max() - row['y']) / (df['y'].max() - df['y'].min()) * (HEIGHT - 1))
        col_idx = int((row['x'] - df['x'].min()) / (df['x'].max() - df['x'].min()) * (WIDTH - 1))
        if 0 <= row_idx < HEIGHT and 0 <= col_idx < WIDTH:
            ground_truth_map[row_idx, col_idx] = row['Radar']
            if not np.isnan(row['prediction']):
                prediction_map[row_idx, col_idx] = row['prediction']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), subplot_kw={'projection': ccrs.PlateCarree()})
    ax1.set_title(f'Ground Truth (Radar) - {valid_dt.strftime("%Y-%m-%d %H:%M")}')
    ax2.set_title(f'Prediction ({model_name}) - {valid_dt.strftime("%Y-%m-%d %H:%M")}')
    for ax, data in [(ax1, ground_truth_map), (ax2, prediction_map)]:
        ax.coastlines()
        ax.add_feature(cfeature.BORDERS)
        ax.gridlines(draw_labels=True)
        im = ax.imshow(data, cmap='Blues', origin='upper', transform=ccrs.PlateCarree(),
                       extent=(df['x'].min(), df['x'].max(), df['y'].min(), df['y'].max()))
        plt.colorbar(im, ax=ax, label='Lượng mưa (mm/h)')
    plt.savefig(os.path.join(output_dir, f'{model_name}_rainfall_map_{valid_dt.strftime("%Y%m%d%H%M")}.png'))
    plt.close()

    plt.figure(figsize=(8, 8))
    plt.scatter(y_test, y_test_pred, alpha=0.5, label='Dữ liệu')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Đường y=x')
    plt.xlabel('Ground Truth (Lượng mưa mm/h)')
    plt.ylabel('Dự đoán (Lượng mưa mm/h)')
    plt.title(f'So sánh Ground Truth và Dự đoán {model_name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f'{model_name}_ground_truth_vs_prediction_scatter.png'))
    plt.close()

    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values(by='importance', ascending=False)

    plt.figure(figsize=(10, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance.head(30))
    plt.title(f'Top 30 Feature Importance từ {model_name}')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.savefig(os.path.join(output_dir, f'{model_name}_feature_importance.png'))
    plt.close()

    return feature_importance, test_rmse


In [6]:
param_sets = [
    {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0},
    {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 1, 'gamma': 0},
    {'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0},
    {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 3, 'gamma': 0.3},
    {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 5, 'gamma': 0.1},
    {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 3, 'gamma': 0.1},
    {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.3, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 1, 'gamma': 0},
]


In [7]:
# Mô tả: Tính tỷ lệ mẫu không mưa/có mưa để xử lý mất cân bằng dữ liệu
y_train_binary = (y_train > 0).astype(int)
n_negative = sum(y_train_binary == 0)
n_positive = sum(y_train_binary == 1)
scale_pos_weight = n_negative / n_positive if n_positive > 0 else 1
print(f"Tỷ lệ mẫu không mưa / có mưa: {scale_pos_weight:.2f}")


Tỷ lệ mẫu không mưa / có mưa: 7.11


In [8]:
# Mô tả: Huấn luyện mô hình với từng bộ tham số, đánh giá, và lưu kết quả
evaluation_results = []
for idx, params in enumerate(param_sets, 1):
    print(f"\nHuấn luyện XGBoost tổ hợp {idx}/{len(param_sets)}")
    xgb_model = XGBRegressor(**params, random_state=42, n_jobs=-1, scale_pos_weight=scale_pos_weight)
    xgb_model.fit(X_train, y_train)
    
    y_train_pred = xgb_model.predict(X_train)
    y_test_pred = xgb_model.predict(X_test)
    
    feature_importance, test_rmse = plot_and_save_results(
        model=xgb_model,
        model_name=f"XGBoost_{idx}",
        params=params,
        y_train_pred=y_train_pred,
        y_test_pred=y_test_pred,
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        df=df,
        output_dir=BEST_MODEL_PLOTS_PATH
    )
    
    if feature_importance is None:
        print(f"Bỏ qua tổ hợp {idx} do không tìm thấy thời điểm hợp lệ")
        continue
    
    y_test_binary = (y_test > 0).astype(int)
    y_pred_binary = (y_test_pred > 0).astype(int)
    print(f"\nĐánh giá nhị phân (Radar = 0 vs Radar > 0) cho XGBoost tổ hợp {idx}:")
    print(classification_report(y_test_binary, y_pred_binary))
    
    evaluation_results.append({
        "model": f"XGBoost_{idx}",
        "params": params,
        "train_rmse": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "test_rmse": test_rmse,
        "train_mae": mean_absolute_error(y_train, y_train_pred),
        "test_mae": mean_absolute_error(y_test, y_test_pred),
        "test_cc": pearsonr(y_test, y_test_pred)[0],
        "test_r2": r2_score(y_test, y_test_pred)
    })



Huấn luyện XGBoost tổ hợp 1/7

Đánh giá XGBoost_1 với tham số {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0}:
Train RMSE: 0.7422, Train MAE: 0.1777
Test RMSE: 0.7987, Test MAE: 0.1814
Test CC: 0.8030
R2 Score (Test): 0.6352


/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)



Đánh giá nhị phân (Radar = 0 vs Radar > 0) cho XGBoost tổ hợp 1:
              precision    recall  f1-score   support

           0       0.99      0.32      0.49     68174
           1       0.17      0.97      0.28      9525

    accuracy                           0.40     77699
   macro avg       0.58      0.65      0.39     77699
weighted avg       0.89      0.40      0.46     77699


Huấn luyện XGBoost tổ hợp 2/7

Đánh giá XGBoost_2 với tham số {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 1, 'gamma': 0}:
Train RMSE: 0.7375, Train MAE: 0.1756
Test RMSE: 0.7997, Test MAE: 0.1796
Test CC: 0.8031
R2 Score (Test): 0.6342

Đánh giá nhị phân (Radar = 0 vs Radar > 0) cho XGBoost tổ hợp 2:
              precision    recall  f1-score   support

           0       0.97      0.31      0.47     68174
           1       0.16      0.94      0.27      9525

    accuracy                           0.38     77699
   mac

In [9]:
# Mô tả: Vẽ ma trận tương quan của các đặc trưng quan trọng và lưu kết quả đánh giá
top_features = feature_importance['feature'].head(30).tolist()
missing_cols = [col for col in top_features if col not in df.columns]
if missing_cols:
    print(f"Cảnh báo: Các cột không có trong df: {missing_cols}")
    top_features = [col for col in top_features if col in df.columns]

correlation_matrix = df[top_features + ['Radar']].corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Ma trận tương quan của top 30 đặc trưng và lượng mưa')
plt.savefig(os.path.join(BEST_MODEL_PLOTS_PATH, 'correlation_matrix.png'))
plt.close()

precip_correlations = correlation_matrix['Radar'].drop('Radar')
print("\nTương quan với lượng mưa (Radar):")
print(precip_correlations.sort_values(ascending=False))

eval_df = pd.DataFrame([
    {
        "model": r["model"],
        "params": str(r["params"]),
        "train_rmse": r["train_rmse"],
        "test_rmse": r["test_rmse"],
        "train_mae": r["train_mae"],
        "test_mae": r["test_mae"],
        "test_cc": r["test_cc"],
        "test_r2": r["test_r2"]
    } for r in evaluation_results
])
eval_df.to_csv(os.path.join(OUTPUT_PATH, 'evaluation_results.csv'), index=False)
print("\nĐã lưu kết quả đánh giá vào evaluation_results.csv")


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1



Tương quan với lượng mưa (Radar):
TCLW    0.213307
TCW     0.193137
R500    0.173187
VSB     0.167600
B04B    0.145432
R850    0.105617
B06B    0.093121
PEV     0.091359
SSHF    0.091236
KX      0.086522
SLHF    0.065484
R250    0.056746
IE      0.055531
B05B    0.006976
CIN     0.000258
ISOR   -0.011342
V250   -0.025946
CAPE   -0.045808
EWSS   -0.053219
hour   -0.055623
SLOR   -0.068709
U250   -0.127304
V850   -0.135154
U850   -0.193968
B10B   -0.204308
B12B   -0.218475
I4B    -0.218969
B16B   -0.221843
B11B   -0.227238
IRB    -0.227383
Name: Radar, dtype: float64

Đã lưu kết quả đánh giá vào evaluation_results.csv


In [10]:
# Mô tả: Tìm và in kết quả của mô hình có test_rmse thấp nhất
if evaluation_results:
    best_result = min(evaluation_results, key=lambda x: x["test_rmse"])
    print("\nKết quả mô hình tốt nhất:")
    print(f"Model: {best_result['model']}")
    print(f"Params: {best_result['params']}")
    print(f"Train RMSE: {best_result['train_rmse']:.4f}, Train MAE: {best_result['train_mae']:.4f}")
    print(f"Test RMSE: {best_result['test_rmse']:.4f}, Test MAE: {best_result['test_mae']:.4f}")
    print(f"Test CC: {best_result['test_cc']:.4f}")
    print(f"Test R2: {best_result['test_r2']:.4f}")
else:
    print("Không có kết quả đánh giá nào được lưu.")


Kết quả mô hình tốt nhất:
Model: XGBoost_7
Params: {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.3, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 1, 'gamma': 0}
Train RMSE: 0.3533, Train MAE: 0.1036
Test RMSE: 0.6439, Test MAE: 0.1392
Test CC: 0.8736
Test R2: 0.7629
